In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
import time
from datetime import date
from bs4 import BeautifulSoup, Comment

In [3]:
team_codes_df = pd.read_csv('team_codes.csv')
games_attended_df = pd.read_csv('sports_attendance.csv')

games_attended_df["URL_Date"] = " "
games_attended_df["Year"] = " "

games_attended_df['Year'] = pd.to_datetime(games_attended_df.Date)
games_attended_df['Year'] = games_attended_df['Year'].dt.strftime('%Y')
games_attended_df['URL_Date'] = pd.to_datetime(games_attended_df.Date)
games_attended_df['URL_Date'] = games_attended_df['URL_Date'].dt.strftime('%Y%m%d0')

nba_attendance_df = games_attended_df[games_attended_df.Sport == ('NBA')].copy()
mlb_attendance_df = games_attended_df[games_attended_df.Sport == ('MLB')].copy()
nfl_attendance_df = games_attended_df[games_attended_df.Sport == ('NFL')].copy()

mlb_df = mlb_attendance_df.merge(team_codes_df, how='left', left_on='Home', right_on='MLB_Teams')
mlb_df = mlb_df.drop(columns=['NBA_Teams', 'NBA_Codes', 'NBA_Teams', 'NFL_Teams', 'NFL_Codes', 'NHL_Teams', 'NHL_Codes', 'MLS_Codes'])

mlb_df['URL_Variable'] = mlb_df['MLB_Codes'] + mlb_df['URL_Date']

away_code_df = mlb_attendance_df.merge(team_codes_df, how='left', left_on='Visitor', right_on='MLB_Teams')
away_code_df = away_code_df.drop(columns=['NBA_Teams', 'NBA_Codes', 'NBA_Teams', 'NFL_Teams', 'NFL_Codes', 'NHL_Teams', 'NHL_Codes', 'MLS_Codes'])

mlb_df['Away_Code'] = away_code_df['MLB_Codes']
mlb_df.rename(columns = {'MLB_Codes':'Home_Code'}, inplace = True)
mlb_df


,Date,Home,Visitor,Sport,URL_Date,Year,MLB_Teams,Home_Code,URL_Variable,Away_Code
0,7/27/2007,San Francisco Giants,Florida Marlins,MLB,200707270,2007,San Francisco Giants,SFN,SFN200707270,FLO
1,7/28/2007,San Francisco Giants,Florida Marlins,MLB,200707280,2007,San Francisco Giants,SFN,SFN200707280,FLO
2,9/5/2008,St. Louis Cardinals,Florida Marlins,MLB,200809050,2008,St. Louis Cardinals,SLN,SLN200809050,FLO
3,9/6/2008,St. Louis Cardinals,Florida Marlins,MLB,200809060,2008,St. Louis Cardinals,SLN,SLN200809060,FLO
4,7/25/2009,Los Angeles Angels,Minnesota Twins,MLB,200907250,2009,Los Angeles Angels,ANA,ANA200907250,MIN
5,7/26/2009,Los Angeles Dodgers,Florida Marlins,MLB,200907260,2009,Los Angeles Dodgers,LAN,LAN200907260,FLO
6,8/22/2009,San Diego Padres,St. Louis Cardinals,MLB,200908220,2009,San Diego Padres,SDN,SDN200908220,SLN
7,4/24/2010,San Francisco Giants,St. Louis Cardinals,MLB,201004240,2010,San Francisco Giants,SFN,SFN201004240,SLN
8,9/26/2010,Arizona Diamondbacks,Los Angeles Dodgers,MLB,201009260,2010,Arizona Diamondbacks,ARI,ARI201009260,LAN
9,10/28/2010,San Francisco Giants,Texas Rangers,MLB,201010280,2010,San Francisco Giants,SFN,SFN201010280,TEX


In [6]:
URL_list = mlb_df["URL_Variable"].values.tolist()
Home_code_list = mlb_df["Home_Code"].values.tolist()
Away_code_list = mlb_df["Away_Code"].values.tolist()
Home_team_list = mlb_df["Home"].values.tolist()
Away_team_list = mlb_df["Visitor"].values.tolist()
Year_list = mlb_df["Year"].values.tolist()
Date_list = mlb_df["URL_Date"].values.tolist()
Actual_date_list = mlb_df["Date"].values.tolist()

In [9]:
coded_home_teams = []

for item in Home_team_list: 
    home_code = re.sub(r'[^a-zA-Z0-9]', '', item)
    coded_home_teams.append(home_code)

coded_home_teams

['SanFranciscoGiants',
 'SanFranciscoGiants',
 'StLouisCardinals',
 'StLouisCardinals',
 'LosAngelesAngels',
 'LosAngelesDodgers',
 'SanDiegoPadres',
 'SanFranciscoGiants',
 'ArizonaDiamondbacks',
 'SanFranciscoGiants',
 'SanFranciscoGiants',
 'SanDiegoPadres',
 'SanFranciscoGiants',
 'LosAngelesDodgers',
 'SanFranciscoGiants',
 'LosAngelesDodgers',
 'LosAngelesDodgers',
 'OaklandAthletics',
 'PhiladelphiaPhillies',
 'NewYorkYankees',
 'LosAngelesAngels',
 'LosAngelesAngels',
 'LosAngelesAngels',
 'LosAngelesDodgers',
 'LosAngelesDodgers',
 'LosAngelesDodgers',
 'SanDiegoPadres',
 'SanFranciscoGiants',
 'ColoradoRockies',
 'MilwaukeeBrewers',
 'ChicagoWhiteSox',
 'ChicagoWhiteSox',
 'MinnesotaTwins',
 'ChicagoCubs',
 'StLouisCardinals',
 'CincinnatiReds',
 'MilwaukeeBrewers',
 'DetroitTigers',
 'StLouisCardinals',
 'StLouisCardinals',
 'ChicagoCubs',
 'PittsburghPirates',
 'KansasCityRoyals',
 'ClevelandGuardians',
 'TorontoBlueJays',
 'StLouisCardinals',
 'StLouisCardinals',
 'StLouis

In [4]:
# getting length of list
length = len(URL_list)

dataframe_dictionary = {}

box_score_df_list = []
home_stats_df_list = []
away_stats_df_list = []
game_logistics_df_list = []

for i in range(length):
    time.sleep(2)
    
    # Create empty lists to hold table data to be scraped
    away_box_score = []
    home_box_score = []
    box_score = []
    
    # input URL and use BeautifulSoup to parse through the page
    url = f'https://www.baseball-reference.com/boxes/{Home_code_list[i]}/{URL_list[i]}.shtml'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    # Grab the table element that has away team statistics
    away_table_placeholder = soup.select_one(f'#all_{FloridaMarlins}batting') 

    # Grab data from table and put it into the list created above
    for tr in away_table.select('tr:has(td)'):
        tds = [td.get_text(strip=True) for td in tr.select('td')]
        away_box_score.append(tds)

    # Grab the table element that has home team statistics
    home_table = soup.select_one(f'#div_box-{Home_code_list[i]}-game-basic') 

    # Grab data from table and put it into the list created above
    for tr in home_table.select('tr:has(td)'):
        tds2 = [td.get_text(strip=True) for td in tr.select('td')]
        home_box_score.append(tds2)

    # Grab the table element that has game box score
    box_score_table = soup.select_one('#all_line_score')

    # Game box score is commented out in html, so this will grab it out of the comments
    for comment in box_score_table.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            table = comment_soup.find("table")

    # Grab data from table and put it into the list created above
    for tr in table.select('tr:has(td)'):
        tds3 = [td.get_text(strip=True) for td in tr.select('td')]
        box_score.append(tds3)
       
    # Create dataframe for away team statistics
    dataframe_dictionary[Away_code_list[i] + Date_list[i]] = pd.DataFrame(away_box_score)

    # Create an empty list to store away team statistics header information
    away_header_list = []

    # Grab the table header information to use as column headers in our away team statistics dataframe
    for tr in away_table.select('tr:has(th)'):
        ths = [th.get_text(strip=True) for th in tr.select('th')]
        away_header_list.append(ths)

    # Create dataframe for home team statistics
    dataframe_dictionary[Home_code_list[i] + Date_list[i]] = pd.DataFrame(home_box_score)

    # Create an empty list to store home team statistics header information
    home_header_list = []

    # Grab the table header information to use as column headers in our home team statistics dataframe
    for tr in home_table.select('tr:has(th)'):
        ths2 = [th.get_text(strip=True) for th in tr.select('th')]
        home_header_list.append(ths2)    
        
    # Create dataframe for game box score
    dataframe_dictionary['Box' + Date_list[i]] = pd.DataFrame(box_score)

    # Create an list to store game box score dataframe header information
    box_score_list = ['1Q', '2Q', '3Q', '4Q', 'T']

    # If a game goes into overtime(s), we need to add to the box score list for each overtime period
    OT_counter = 1
    while len(box_score[0]) > len(box_score_list):
        box_score_list.insert(-1, f'{OT_counter}OT')
        OT_counter = OT_counter + 1
        
    # Update the column and row labels
    dataframe_dictionary['Box' + Date_list[i]] = dataframe_dictionary['Box' + Date_list[i]].set_axis(box_score_list, axis=1, inplace=False)
    dataframe_dictionary['Box' + Date_list[i]] = dataframe_dictionary['Box' + Date_list[i]].set_axis([f'{Away_code_list[i]}', f'{Home_code_list[i]}'])
    
    dataframe_dictionary['Box' + Date_list[i]]['Date'] = Actual_date_list[i]
    box_score_df_list.append(dataframe_dictionary['Box' + Date_list[i]])
    
    # Pull the away team player data and combine the lists into one list
    away_starters = away_header_list[2:7]
    away_bench = away_header_list[8:-1]
    away_players = [away_starters,away_bench]

    # Pull the home team player data and combine the lists into one list
    home_starters = home_header_list[2:7]
    home_bench = home_header_list[8:-1]
    home_players = [home_starters,home_bench]

    # Consolodate the away team list of lists into one list
    away_starters_list = [item for sublist in away_starters for item in sublist]
    away_bench_list = [item for sublist in away_bench for item in sublist]
    away_roster = away_starters_list + away_bench_list

    # Consolodate the home team list of lists into one list
    home_starters_list = [item for sublist in home_starters for item in sublist]
    home_bench_list = [item for sublist in home_bench for item in sublist]
    home_roster = home_starters_list + home_bench_list

    # Add Team Totals to the end of the away team list
    away_roster.append('Team Totals')
    dataframe_dictionary[Away_code_list[i] + Date_list[i]].index = away_roster

    # Add Team Totals to the end of the home team list
    home_roster.append('Team Totals')
    dataframe_dictionary[Home_code_list[i] + Date_list[i]].index = home_roster

    # Set the home and away dataframe column labels
    column_headers = away_header_list[1]
    column_headers.remove("Starters")
    dataframe_dictionary[Away_code_list[i] + Date_list[i]].columns = column_headers
    dataframe_dictionary[Home_code_list[i] + Date_list[i]].columns = column_headers

    dataframe_dictionary[Away_code_list[i] + Date_list[i]]['Date'] = Actual_date_list[i]
    dataframe_dictionary[Home_code_list[i] + Date_list[i]]['Date'] = Actual_date_list[i]
    
    dataframe_dictionary[Away_code_list[i] + Date_list[i]]['Team'] = Away_code_list[i]
    dataframe_dictionary[Home_code_list[i] + Date_list[i]]['Team'] = Home_code_list[i]
    
    home_stats_df_list.append(dataframe_dictionary[Home_code_list[i] + Date_list[i]])
    away_stats_df_list.append(dataframe_dictionary[Away_code_list[i] + Date_list[i]])
    
    # Pull logistical information about the game: attendance, length of game, and venue
    text = soup.get_text()
    attendance_string = re.findall(r"Attendance:\s+\d*,\d*", text)
    
    # A failsafe, as one of the games has no attendance listed causing the loop to break
    if(len(attendance_string) != 0):
        
        # Clean up the attendance string that is pulled 
        new_attendance_string = attendance_string[0].strip('([^a-z]xa)')
        attendance_list = list(new_attendance_string)
        attendance_list.pop(11)
    else:
        attendance_string = "No Attendance Available"
    
    # Clean up the game length string that is pulled 
    game_time_string = re.findall(r"Time of Game:\s+\d:\d*", text)    
    new_game_time_string = game_time_string[0].strip('([^a-z]xa)')
    game_time_list = list(new_game_time_string)
    game_time_list.pop(13)

    # join the lists to create final string
    final_attendance_string = "".join(attendance_list)
    final_game_time_sting = "".join(game_time_list)

    # Create throwaway variables to hold split strings so we can pick out what we want later
    x = final_attendance_string.split(":")
    y = final_game_time_sting.split(":",1)

    # Pick out the strings before the : that we split above
    dataframe_dictionary['Attendance' + Date_list[i]] = pd.DataFrame(x, columns = [x[0]])
    dataframe_dictionary['Attendance' + Date_list[i]] = dataframe_dictionary['Attendance' + Date_list[i]].drop([0])
    dataframe_dictionary['Game_Time' + Date_list[i]] = pd.DataFrame(y, columns = [y[0]])
    dataframe_dictionary['Game_Time' + Date_list[i]] = dataframe_dictionary['Game_Time' + Date_list[i]].drop([0])

    # Join the attendance and game time dataframes into one dataframe
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = pd.concat([dataframe_dictionary['Attendance' + Date_list[i]], dataframe_dictionary['Game_Time' + Date_list[i]]], ignore_index=True, sort=False)
    dataframe_dictionary['Game_Logistics' + Date_list[i]]['Time of Game'] = dataframe_dictionary['Game_Logistics' + Date_list[i]]['Time of Game'].shift(-1)
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = dataframe_dictionary['Game_Logistics' + Date_list[i]].dropna()

    # Scrape the name of the vanue and location and add that to the logistics dataframe
    divparent = soup.find('div', attrs={'class':'scorebox_meta'})
    divparent.text
    text = divparent.text
    result = re.split(r'\n', text)
    almost_done = result[1]
    final_result = almost_done.partition(f'{Year_list[i]}')[2]
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = dataframe_dictionary['Game_Logistics' + Date_list[i]].assign(Venue=[final_result])

    dataframe_dictionary['Game_Logistics' + Date_list[i]]['Date'] = Actual_date_list[i]
    game_logistics_df_list.append(dataframe_dictionary['Game_Logistics' + Date_list[i]])
    

In [10]:
########## GUIDE TO ACCESSING DATAFRAMES ##########

## Pull Home Stats --> dataframe_dictionary['(Home Code)(Date)']

## Pull Away Stats --> dataframe_dictionary['(Away Code)(Date)']

## Pull Box Score --> dataframe_dictionary['Box(Date)']

## Pull Game Logistics --> dataframe_dictionary['Game_Logistics(Date)']

In [11]:
#Test
dataframe_dictionary['Game_Logistics201812020']

,Attendance,Time of Game,Venue,Date
0,"19,600",2:14,"AmericanAirlines Arena, Miami, Florida",12/2/2018


In [12]:
#Test
dataframe_dictionary['UTA201812020']

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date,Team
Donovan Mitchell,37:40,8,24,.333,1,7,.143,1,1,1.000,...,5,3,1,0,2,2,18,-5,12/2/2018,UTA
Rudy Gobert,37:15,5,9,.556,0,0,,2,5,.400,...,18,4,1,3,2,2,12,+19,12/2/2018,UTA
Ricky Rubio,36:05,9,13,.692,2,4,.500,3,3,1.000,...,5,6,1,0,3,3,23,+14,12/2/2018,UTA
Jae Crowder,34:09,6,15,.400,3,7,.429,4,4,1.000,...,6,2,1,0,4,3,19,+1,12/2/2018,UTA
Joe Ingles,29:10,5,13,.385,1,4,.250,0,0,,...,4,4,2,0,0,4,11,+7,12/2/2018,UTA
Royce O'Neale,18:43,2,2,1.000,2,2,1.000,0,0,,...,2,1,0,1,1,1,6,+4,12/2/2018,UTA
Derrick Favors,17:53,2,4,.500,1,2,.500,0,0,,...,7,0,2,2,0,2,5,-19,12/2/2018,UTA
Kyle Korver,17:23,1,6,.167,1,5,.200,0,0,,...,2,0,0,0,1,0,3,-18,12/2/2018,UTA
Thabo Sefolosha,6:39,0,1,.000,0,1,.000,0,0,,...,1,0,0,0,0,0,0,-5,12/2/2018,UTA
Raul Neto,3:07,1,1,1.000,0,0,,0,0,,...,0,1,0,0,0,0,2,-4,12/2/2018,UTA


In [13]:
#Test
dataframe_dictionary['MIA201812020']

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date,Team
Josh Richardson,39:10,4,16,.250,2,6,.333,6,6,1.000,...,3,4,0,0,4,2,16,-13,12/2/2018,MIA
Wayne Ellington,37:12,4,13,.308,4,9,.444,0,0,,...,7,1,1,0,0,4,12,-2,12/2/2018,MIA
Hassan Whiteside,34:02,11,17,.647,0,0,,1,6,.167,...,20,2,1,3,1,4,23,-8,12/2/2018,MIA
James Johnson,18:46,4,10,.400,1,2,.500,0,0,,...,2,2,1,1,2,2,9,-16,12/2/2018,MIA
Rodney McGruder,5:07,0,1,.000,0,0,,0,0,,...,0,0,0,0,0,0,0,-14,12/2/2018,MIA
Justise Winslow,32:05,4,9,.444,1,2,.500,0,0,,...,5,1,2,1,4,3,9,+14,12/2/2018,MIA
Dwyane Wade,30:26,4,15,.267,2,6,.333,5,5,1.000,...,3,8,1,1,1,0,15,+21,12/2/2018,MIA
Kelly Olynyk,29:10,4,8,.500,3,6,.500,5,7,.714,...,8,1,1,1,1,0,16,+18,12/2/2018,MIA
Bam Adebayo,14:01,1,2,.500,0,0,,0,0,,...,2,1,1,1,0,1,2,+10,12/2/2018,MIA
Tyler Johnson,Did Not Dress,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,12/2/2018,MIA


In [14]:
#Test
dataframe_dictionary['Box201812020']

,1Q,2Q,3Q,4Q,T,Date
UTA,35,20,22,23,100,12/2/2018
MIA,27,26,27,22,102,12/2/2018


In [15]:
total_box_score_df = pd.concat(box_score_df_list)

total_box_score_df

,1Q,2Q,3Q,4Q,T,Date,1OT
UTA,16,32,20,28,96,2/11/2014,NaN
LAL,27,10,23,19,79,2/11/2014,NaN
ATL,31,26,25,16,98,1/2/2015,NaN
UTA,21,24,23,24,92,1/2/2015,NaN
MEM,30,26,29,22,107,4/3/2016,NaN
ORL,29,38,30,22,119,4/3/2016,NaN
UTA,24,23,22,32,101,12/27/2017,NaN
GSW,23,25,42,36,126,12/27/2017,NaN
LAL,31,22,28,36,117,11/17/2018,NaN
ORL,25,37,38,30,130,11/17/2018,NaN


In [16]:
total_home_stats_df = pd.concat(home_stats_df_list)

team_col = total_home_stats_df.pop("Team")
total_home_stats_df.insert(0, "Team", team_col)

total_home_stats_df

,Team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date
Wesley Johnson,LAL,37:21,7,14,.500,1,3,.333,0,0,...,6,6,0,0,0,2,3,15,-14,2/11/2014
Chris Kaman,LAL,36:44,11,24,.458,0,0,,3,3,...,10,14,4,1,3,3,3,25,-5,2/11/2014
Steve Blake,LAL,36:25,2,11,.182,1,4,.250,0,0,...,3,5,8,2,0,1,2,5,-3,2/11/2014
Shawne Williams,LAL,36:13,4,8,.500,2,4,.500,1,2,...,5,6,0,0,2,0,3,11,-10,2/11/2014
Steve Nash,LAL,16:35,1,4,.250,0,2,.000,0,0,...,1,1,2,0,0,0,0,2,-6,2/11/2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Terence Davis,SAC,Did Not Play,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3/25/2023
PJ Dozier,SAC,Did Not Play,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3/25/2023
Keon Ellis,SAC,Did Not Play,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3/25/2023
Alex Len,SAC,Did Not Play,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3/25/2023


In [17]:
total_away_stats_df = pd.concat(away_stats_df_list)

team_col = total_away_stats_df.pop("Team")
total_away_stats_df.insert(0, "Team", team_col)

total_away_stats_df

,Team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Date
Gordon Hayward,UTA,34:46,5,13,.385,2,4,.500,3,4,...,8,8,7,2,0,4,2,15,+12,2/11/2014
Trey Burke,UTA,28:49,3,11,.273,1,3,.333,1,1,...,3,4,8,1,0,1,1,8,+2,2/11/2014
Derrick Favors,UTA,27:03,4,9,.444,0,0,,3,7,...,8,10,1,1,3,1,3,11,-4,2/11/2014
Marvin Williams,UTA,22:29,4,8,.500,1,2,.500,0,0,...,4,7,1,0,0,1,0,9,-4,2/11/2014
Richard Jefferson,UTA,19:53,0,4,.000,0,2,.000,2,2,...,1,1,2,0,0,0,0,2,-13,2/11/2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Udoka Azubuike,UTA,10:22,1,2,.500,0,0,,0,0,...,3,6,1,0,0,1,0,2,-4,3/25/2023
Johnny Juzang,UTA,9:33,1,5,.200,0,3,.000,0,0,...,1,2,0,0,0,0,0,2,-4,3/25/2023
Damian Jones,UTA,7:23,0,0,,0,0,,0,0,...,1,2,1,1,1,0,0,0,+4,3/25/2023
Micah Potter,UTA,Did Not Play,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3/25/2023


In [18]:
total_game_logistics_df = pd.concat(game_logistics_df_list)
total_game_logistics_df

,Attendance,Time of Game,Venue,Date
0,"18,209",1:59,"STAPLES Center, Los Angeles, California",2/11/2014
0,"19,029",2:09,"EnergySolutions Arena, Salt Lake City, Utah",1/2/2015
0,"17,741",2:08,"Amway Center, Orlando, Florida",4/3/2016
0,"19,596",2:11,"Oracle Arena, Oakland, California",12/27/2017
0,"19,249",2:10,"Amway Center, Orlando, Florida",11/17/2018
0,"15,898",2:19,"Amway Center, Orlando, Florida",11/18/2018
0,"19,600",2:14,"AmericanAirlines Arena, Miami, Florida",12/2/2018
0,"16,447",2:06,"State Farm Arena, Atlanta, Georgia",11/8/2019
0,"16,447",2:10,"Golden 1 Center, Sacramento, California",11/3/2021
0,"12,993",2:15,"Golden 1 Center, Sacramento, California",11/7/2021


In [19]:
index_list = total_box_score_df.index.tolist()
home_away_list = []


for i in range(len(index_list)):
    if (i % 2) == 0:
        home_away_list.append('Away')
    else:
        home_away_list.append('Home')


total_box_score_df = total_box_score_df.set_index('Date')
total_box_score_df.insert(0, 'Team', index_list)
total_box_score_df.insert(0, 'Home_Away', home_away_list)
total_box_score_df

,Home_Away,Team,1Q,2Q,3Q,4Q,T,1OT
Date,,,,,,,,
2/11/2014,Away,UTA,16,32,20,28,96,NaN
2/11/2014,Home,LAL,27,10,23,19,79,NaN
1/2/2015,Away,ATL,31,26,25,16,98,NaN
1/2/2015,Home,UTA,21,24,23,24,92,NaN
4/3/2016,Away,MEM,30,26,29,22,107,NaN
4/3/2016,Home,ORL,29,38,30,22,119,NaN
12/27/2017,Away,UTA,24,23,22,32,101,NaN
12/27/2017,Home,GSW,23,25,42,36,126,NaN
11/17/2018,Away,LAL,31,22,28,36,117,NaN


In [20]:
total_game_logistics_df.to_csv('total_game_logistics.csv')
total_away_stats_df.to_csv('total_away_stats.csv')
total_home_stats_df.to_csv('total_home_stats.csv')
total_box_score_df.to_csv('total_box_score_df.csv')
# hof_df.to_csv('nba_hof.csv')
# mvp_df.to_csv('nba_mvp.csv')
# all_league_df.to_csv('all_league_players.csv')
# allstar_df.to_csv('allstars.csv')
# agg_dpoy_seen_df.to_csv('dpoy.csv')